# Table of contents
1. [Introduction](#introduction)
2. [QuickStart](#quickstart)
3. [Modelling](#modelling)
    1. [Variational Autoencoder](#vae)
    2. [Model architecture](#architecture) 
    3. [Encoder](#encoder)
    4. [Decoder](#decoder)
    5. [VariationalAutoEncoder](#vaecode)
    6. [Autoencoder](#autoencoder)
    7. [Factory class](#factory) 
4. [Evaluation](#eval)
5. [Discussion](#discuss)


# Introduction <a name="introduction"></a>

In this notebook we consider the task of developing a generative model for creating synthetic data. We briefly examine the main object ```Factory``` in the ```aidatafactory``` library which is to be able to generate artificial data while preserving the statistical properties of the real data.

In [ ]:
# install libraries
!pip install pandas -q gwpy
!pip install aidataFactory -q gwpy
!pip install matplotlib==3.1.3 -q gwpy

In [ ]:
# load libraries
import pandas as pd
from aidatafactory import Factory

# Quickstart <a name="quickstart"></a>
In this section we run quickly through the functionality of the <b>Factory</b> class and generate synthetic data. This can take up to 8 minutes. Further below you can read more details about implementation.

#### Step 1. Run the cell below to load data.


In [ ]:
import os
url = 'https://github.com/ngocuong0105/dataFactory/blob/master/showcase/income_data.csv'
income_data = pd.read_csv(url, error_bad_lines=False); income_data

#### Step 2. Instantiate Factory object. Need to specify numerical and categorical columns.

In [ ]:
factory = Factory(columns_spec = {'numerical':['Income'], 'categorical':['Gender']})

#### Step 3. Train generative model.
Pregress of fitted models for each category are plotted.

In [ ]:
factory.learn(income_data)

#### Step 4. Generate as many as you wish artificial samples.

In [ ]:
ai_data = factory.generate(num_rows = len(income_data), data = income_data);ai_data

#### Step 5. Compare distributions.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
for gender in income_data['Gender'].unique():
    fig = plt.figure(figsize=(7,4))
    real = income_data[income_data['Gender']==gender]
    synth = ai_data[ai_data['Gender']==gender]
    sns.histplot(real['Income'], x = real['Income'], element="poly",  stat = 'density',alpha = 1)
    sns.histplot(synth['Income'], x = synth['Income'], element="poly",  stat = 'density', color= 'purple', alpha = 0.5)
    fig.legend(labels=['Real','Artificial'])
    plt.title('Real vs Artificial')
    plt.xlim(synth['Income'].quantile(0.05), synth['Income'].quantile(0.95))
    plt.show()

# Modelling <a name="modelling"></a>
In the next sections we describe the mathematics behind the output <b>a distribution</b> which will model the real data as closely as possible and we show how we use this distribution to sample synthetic data points. We make a key assumption about the dataset:
- assume that all rows in the table are independent

Note for time-series datases or those where there is conditional dependence between rows this assumtion would be violated. Typically in literature (and practice) the assumption reads "independent and identically distributed (i.i.d.)", however, here we ommited the identical distribution part as we saw that for different effort categories we have a different distribution (normal, bernoulli, mixed).

In the next few subsections we describe a well-known machine-learning model Variational Autoencoder and discuss why it is suitable for our task of learning a distribution for creating synthetic data from our real dataset.

### Autoencoder <a name="autoencoder"></a>
An autoencoder is a neural network that is trained to attempt to copy its input
to its output (almost what we want). It has a hidden layer that describes a code used to
represent the input in a lower dimensional latent representation. Then the autoencoder decodes the latent representation back to a reconstructed term. The network may be viewed as consisting of two parts: an encoder function $h = f(x)$ and a decoder that produces a reconstruction term $r = g(h)$. If an autoencoder succeeds in simply
learning to set $g(f (x)) = x$ everywhere (i.e. the identity function), then it is not especially useful. Instead, autoencoders are designed to be unable to learn to copy perfectly (this is what we want).


### Variational Autoencoder <a name="vae"></a>
For our task recall that we need as an output a generative model from which we would sample synthetic data. Using just an autoencoder is not enough since we would learn the functions $f$ and $g$ which are deterministic. We need stachastic mappings versions of $f$ and $g$ and this naturally extends to the use of Variatioonal Autoencoders also known as VAE - our choice of architecture.

A VAE is a probabilistic take on the autoencoder, a model which takes high dimensional input data and compresses it into a smaller representation. Unlike a traditional autoencoder, which maps the input onto a latent vector, a VAE maps the input data into the parameters of a probability distribution, such as the mean and variance of a Gaussian. Using it we will learn to compress the data while minimizing the reconstruction error. The two main reasons why we choose to work with VAE for this task are:

- Simplicity. The encoding and decoding operations in VAE naturally allow us to create synthetic data.
- Bayesian inference. We are allowed to put priors on the encoder and decoder which makes the model customizable to each dataset. We will log-transform our data and set normal priors as we saw the log data follows normal.

### Model architecture: <a name="architecture"></a>
VAE has 2 main parts - encoder and decoder. We define for each data item $i = 1,2,3,...,n:$
- Let $Z_i$ be a (latent) $k$-dimensional normally distributed random vector with mean
0 and identity covariance: $Z_i ∼ N(0,I_k)$
- Given $Z_i$ , the distribution of the $i$-th data item is a $p$-dimensional nonlinear and non-Gaussian: $X_i |Z_i ∼ p_\theta(·|Z_i )$, where  $p_\theta(·|Z_i )$ is the <b>decoder</b>.
For example, $p_\theta(·|Z_i)$ can be a Gaussian with mean $\mu_\theta(Z_i)$ and diagonal variance $\sigma^2_\theta (Z_i)$,
where both $\mu_\theta$ and $\sigma^2_\theta$ are outputs of a neural network with parameters $\theta$. For another
example, $p_\theta(X_i|Z_i)$ can be an independent Bernoulli for each element of $X_i$ , with mean $\mu_\theta(Z_i)\in(0, 1)$, with the range restricted to $(0, 1)$ using a <b>sigmoid</b> output nonlinearity. All these possibilities for the decoder are added as a functionality in the <b>Decoder</b> class. Note that if we choose simple linear function for $p_\theta(·|Z_i )= \mu+\sigma Z_i$, we will have probabilistic PCA which is a basic linear dimensionality reduction technique. 
- Given the neural network in the generative model, the exact posterior
distribution $p(z|x)$ will typically not be tractable anymore. Instead we consider a variational
distribution $q_\phi(z|x)$, called a recognition model or <b>encoder</b>, which is itself given by a
neural network with parameters φ. The encoder is typically a Gaussian with a diagonal
covariance matrix, $q_\phi(z|x) = N (z; \mu_\phi(x), \sigma_\phi(x))$

Next we define a metric which we would optimise while training our model.

<b>Definition</b>. Variational free energy in a latent variable model $p(x, z|\theta)$ is defined as $F(\theta, q) = E_q (log p(x, z|\theta) − log q(z))$. It is a natural lower bound of the log-likelihood (evidence): 

log$p(x|θ) ≥ E_{z∼q}$ log$\frac{p(x,z|\theta)}{q(z)} = E_{z∼q}$ log$ p(x, z|\theta) − E_{z∼q} $log $q(z)$

The variational free energy is also known as ELBO (evidence lower bound).

In our notebook we will minimize the difference between the likelihood and the ELBO. Other implementations might maximize ELBO. Finally, we will consider separate model for each type of 'effort' because we saw that they have different distributions, as a result we develop a separate Variational Autoencoder for each 'effort' category .

### Coding architecture
We will build our own VariationalAutoencoder using the tenserflow.keras model subclassing API. We choose this architecture as it is the most flexible option allowing us to build fully-customizable models. Also this architecture follows closely the OOP paradigm which (imo) enhances code readability

### Encoder <a name="encoder"></a>
The encoder class encapsulates the functionality of the variational distribution or recognition model $q_\phi(z|x) = N (z; \mu_\phi(x), \sigma_\phi(x))$. We define 2 shallow (one for the mean and one for the variance) feed-forward neural networks each with one layer. In reality, we do not create neural network for the variance but rather for the log-variance. This is because variance has to be strictly positive whereas log-variance can be both negative and positive, making it easier to model with a neural network without the need for additional constraints.

The ELBO objective can be rewritten (see VariationalAutoEncoder section for derivation):

ELBO = $E_{q_\phi(z|x)}($ log $p_\theta(x|z)) − KL (q_\phi(z|x)||p(z))$,

where the second therm denotes KL-divergence (measure of distance between distributions).

During training we need to estimate the first term $E_{q_\phi(z|x)}($ log $p_\theta(x|z))$ which requires sampling from $q_\phi(z|x)$. Getting a good Monte Carlo estimator would require drawing many samples of codes for each obervation $x_i$ (i.e sampling from decoder) , which is prohibitive and slow. A simple solution to this is the reparametrization trick. For example, in the case of a normal variational posterior, a draw $z_i ∼ N (z; \mu_\phi(x), \sigma_\phi(x))$
can be written as $z_i = \mu_\phi(x) + \sigma_\phi(x) \epsilon$, with $\epsilon ∼ N (0, I)$. The reparametrization function is available in the <b>Sampling</b> class.

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras import Model
from tensorflow.keras import layers, initializers

In [ ]:
class Encoder(layers.Layer):
    """ 
    Maps input data tensor to latent triple vector (z_mean, z_log_var, z)
    """

    def __init__(self, latent_dim: int, name:str = "encoder", **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_mean = layers.Dense(latent_dim, bias_initializer='ones') # linear activations
        self.dense_log_var = layers.Dense(latent_dim, kernel_initializer = initializers.RandomNormal(stddev=0.1)) # use logvar as var need constaint to be positive
        self.sampling = Sampling()

    def call(self, inputs: tf.Tensor):
        x = inputs
        z_mean = self.dense_mean(x) # 
        z_log_var = self.dense_log_var(x)
        z = self.sampling.reparametrization((z_mean, z_log_var))
        return z_mean, z_log_var, z


### Decoder <a name="decoder"></a>
The decoder class encapsulates the functionality of the output of the generative model or decoder distribution $p_\theta(·|Z_i )$. Its goal is to capture any non-linear relationship and reconstruct the compressed data.
Our Decoder class supports 4 different decoding distributions:
1. Simple relu decoder - one layer neural network with relu activation, we zeroe out negative values as MonthlyIncome is non-negative.
2. Normal decoder - one layer for the mean and one layer for the log-variance.
3. Bernoulli decoder - one layer for the mean (added for 0-1 outputs).
4. Gamma decoder <b>(under development - manual hypertuning recommended )</b> - one layer for the shape parameter and one for the rate parameter. 



In [ ]:
class Decoder(layers.Layer):
    """
    Converts z, the encoded tensor, back into a reconstructed term.
    """

    def __init__(self, original_dim: int, distribution: str = 'default', name: str = 'decoder', **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.distribution = distribution
        self.sampling = Sampling()

        # simple relu decoder (default)
        self.dense_output = layers.Dense(original_dim, activation = 'relu') # dont want negative values

        # normal  decoder
        self.dense_mean = layers.Dense(original_dim, bias_initializer='ones') # linear activations
        self.dense_log_var = layers.Dense(original_dim, kernel_initializer = initializers.RandomNormal(stddev=0.1))
        
        # bernoulli decoder
        self.bernoulli_lambda = layers.Dense(original_dim, activation = 'sigmoid')

        # gamma decoder
        self.gamma_log_alpha = layers.Dense(original_dim, bias_initializer='ones', kernel_initializer = initializers.RandomNormal(stddev=0.1)) # use logvar as var need constaint to be positive
        self.gamma_log_beta = layers.Dense(original_dim, bias_initializer='zeros', kernel_initializer = initializers.RandomNormal(stddev=0.1)) # use logvar as var need constaint to be positive
        
    def call(self, inputs: tf.Tensor):
        if self.distribution == 'default':
            return self.dense_output(inputs)

        elif self.distribution == 'normal':
            z = inputs
            mean = self.dense_mean(z)
            log_var = self.dense_log_var(z)
            output = self.sampling.normal((mean,log_var))
            return output

        elif self.distribution == 'bernoulli':
            z = inputs
            mean = self.bernoulli_lambda(z)
            output = self.sampling.bernoulli(mean)
            output = tf.cast(output, tf.float32) 
            return output
            
        elif self.distribution == 'gamma':
            z = inputs
            log_alpha = self.gamma_log_alpha(z)
            log_beta = self.gamma_log_beta(z)
            output = self.sampling.gamma((log_alpha,log_beta))
            return output


### VariationalAutoEncoder <a name="vaecode"></a>
VariationalAutoEncoder class provides end-to-end interface for encoding and decoding into one place. In the call method we add the KL-divergence as the ELBO bound can be rewritten as:

ELBO =  $E_q ($log $p(x, z|\theta) − $log$ q(z)) $<br>$
     = E_q ($log$ p(x|z,\theta) + E_q ($log$ p(z)) − E_q ($log$ q(z|x)) $ <br>
         $ = E_{q_\phi(z|x)}($ log $p_\theta(x|z)) − KL (q_\phi(z|x)||p(z))$
         
The first term is the reconstruction error and the second term is the KL-distance between the true encoding distribution $p(z)$ and the variational distribution  $q_\phi(z|x)$ which estimates the encoder.

In [ ]:
class VariationalAutoEncoder(Model):
    """
    Combines the encoder and decoder into one model for training.
    """

    def __init__(
        self,\
        original_dim: int,\
        latent_dim: int,\
        distribution: str,\
        name:str = "variational autoencoder",\
        **kwargs\
    ):
        super(VariationalAutoEncoder, self).__init__(name = name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim = latent_dim)
        self.decoder = Decoder(original_dim, distribution = distribution)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs) 
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

### Factory <a name="factory"></a>

The Factory class is a little bit longish, so we will not attach it here. Please refer to it in the [git repository](https://github.com/ngocuong0105/dataFactory) 
Recall that in the QuickStart we instantiated Factory object like that:


In [ ]:
factory = Factory(columns_spec = {'numerical':['Income'], 'categorical':['Gender']})

Infact the class <b>Factory</b> supports more arguments which could come in handy when experimenting different models and fine-tuning parameters:

In [ ]:
factory = Factory(
                columns_spec = {'numerical':['Income'], 'categorical':['Gender']},
                distribution_spec = {'Male':'gamma','Female':'noemal'},
                log_transform = True,
                latent_dim = 32,
                batch_size = 64,
                epochs = 15,
                learning_rate = 1e-3
            )
factory.learn(income_data)

Key parameter which we have <b> not </b> used in the QuickStart is <b>distribution_spec</b>. Our generative model have assumed independent distributions between rows in the table but allows non-identical distributions. <b>distribution_spec</b> is a dictionary where the keys are the category values (Male or Female category) and the dictionary values are the decoder distributions ('default','bernoulli','normal','gamma' - see Decoder section for more information). Above we assign <b>distribution_spec</b> in Factory to have gamma distribution Income for Male and normal distribution for Female.

Next generate synthetic samples:

In [ ]:
ai_data = factory.generate(len(income_data), income_data)

# Evaluation <a name="eval"></a>

The evaluation of the generated artificial data can be assessed by evaluating the effectiveness of machine learning tasks. Models that are trained on the artificial data can be compared with models trained on the original data, and scored on criteria such as accuracy metrics. In our particular problem we have not defined concrete machine learning task - thus we will simply compare the plots and descriptive of the original and artificial datasets.

Consider the plot comparision between artificial and real data overall:

In [ ]:
income_data.groupby('Gender').describe()

In [ ]:
ai_data.groupby('Gender').describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
for gender in income_data['Gender'].unique():
    fig = plt.figure(figsize=(7,4))
    real = income_data[income_data['Gender']==gender]
    synth = ai_data[ai_data['Gender']==gender]
    sns.histplot(real['Income'], x = real['Income'], element="poly",  stat = 'density',alpha = 1)
    sns.histplot(synth['Income'], x = synth['Income'], element="poly",  stat = 'density', color= 'purple', alpha = 0.5)
    fig.legend(labels=['Real','Synthetic'])
    plt.title('Real vs Synthetic')
    plt.xlim(synth['Income'].quantile(0.05), synth['Income'].quantile(0.95))
    plt.show()

Successful evaluation metrics need to choose should punish if the original data is really different from the synthetic data and also punish if they are too similar as identical datasets are useless.  Our goal is to measure synthetic data effectiveness when in use compared to the original data. There are several groups of evaluation metrics:

- Statistical metrics: These metrics compare the statistical properties of the synthetic and original datasets. Includes histogram plots, descriptive statistics analysis, hypothesis testing for distributional difference.

- Predictive power metrics: These metrics compare the performance of Machine Learning alforithm trained on synthetic and on original datasets. 

- Likelihood metrics: These metrics fit probabilistic model to real data and estimate the chance that the synthetic data comes from this distribution.

- Detection metrics: These metrics try to train a Machine Learning Classifier that learns to distinguish the real data from the synthetic data, and report a score of how successful this classifier is.


# Discussion <a name="discuss"></a>

We have created our own generative model Variational Autoencoder with which we sampled artificial data. From the
comparison plots of the artificial and original data distributions above we see that the artificial data is somewhat more monotone and systemically has lower mean. More experiments on the initialization of parameters of the encoder and decoder are needed as so far we have only eye balled what the correct initialization should be. Also the real data set is only with integers, so rounding our results could be helpful.  As for evaluation, without having a specific machine learning task (e.g. regression, classification) we cannot use predictive metrics to quantify how good is our artificial dataset. However, if we have a certain task then a good idea for evaluation would be to benchmark our artificial data generation against the original dataset added with some Gaussian noise. 

Finally, it is worth to note that although VAE is simple and easy to understand, it cannot capture the case of dependent rows in data (e.g time-series). Also Variational Autoencoders and its extensions suffer from scalability issues. It mainly results from the costly decoding operations involved in the graph reconstruction where we multiply dense matrices. Therefore generating artificial data when working with large databases and multiple tables can be an issue. For timely structured data synthesis literature suggests usage of LSTM and for better scalability utilization of other directed and undirected graphical models such as GANs.
